In [1]:
import requests  # 서버에 http 프로토콜로 요청
from bs4 import BeautifulSoup as bs  # html 파싱
import time  # 5초간 휴식 할 때 사용
from datetime import datetime  # 오늘 연월일 추출
import pandas as pd  # 데이터 프레임 만듬
from tqdm import tqdm
import sqlite3
from sqlalchemy import create_engine
import yes24_dbio as ydb

In [ ]:
pwd

In [2]:
today = datetime.today()
yyyy = today.year
mm = today.month
print(yyyy, mm)

2023 12


In [3]:
year = list(range(2023, 2007, -1))
month = list(range(1, 13))
page = list(range(1, 10))
print(year)
print(month)
print(page)


[2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
help(ydb.extract_bookinfo)

In [ ]:
result = []
for yy in year:
    for mon in month if yy != yyyy else range(1, mm):
        for p in page:
            print(f"{yy}년{mon}월{p}페이지 수집중", end="\r")
            url = "https://www.yes24.com/Product/Category/MonthWeekBestSeller"
            payload = dict(categoryNumber='001', pageNumber=p, pageSize=120, type='month', saleYear=yy, saleMonth=mon)
            
            try:
                r = requests.get(url, params=payload) # yes24 서버에 url로 자료 요청
            except Exception as e:
                print(e)
            
            soup = bs(r.text, 'lxml')   # yes24 서버에서 받은 응답 값 텍스트를 beautifulsoup을 통해 html로 파싱
            book_list = soup.select("#yesBestList > li")
            result.append(ydb.extract_bookinfo(yy, mon, book_list))  # extract_bookinfo 함수로 추출한 데이터를 result에 추가
            time.sleep(5)
            print(end="\r")



In [ ]:
# 추출한 베스트셀러 데이터를 데이터프레임으로 변환해서 합침
best_total = pd.DataFrame()
for best_list in result:
    temp = pd.DataFrame(best_list, columns=['yy', 'mon', 'title_h', 'title_f', 'title_m', 'title_e', 'detail_link', \
                                    'author', 'publisher', 'pub_date', 'price', 'n_reviews', \
                                     'review_link', 'rating', 'tags'])
    best_total = pd.concat([best_total, temp])
best_total.to_csv(f"./data/yes24/new_yes24bestseller{year[-1]}{month[0]}_{year[0]}{month[-1]}_1000.csv", index=False)

In [ ]:
# 저장된 파일에서 데이터를 불러와 세부 데이터 추출
data2 = pd.read_csv(f"./data/yes24/new_yes24bestseller{year[-1]}{month[0]}_{year[0]}{month[-1]}_1000_v2.csv")
urls = data2['detail_link']
urls = list(set(urls)) # 중복된 URL을 제거하고 수집
detail_page_data = ydb.detail_page_info(urls)  # detail_page_info 함수로 상세페이지 데이터 추출
best_total['book_id'] = best_total['detail_link'].apply(lambda x: x.split("/")[-1]) 
final_result_df = pd.merge(best_total, detail_page_data, how='inner', on="book_id") # 베스트 셀러 목록과 detail info 합침
final_result_df.head(2)
ydb.to_db(final_result_df) # 최종 데이터를 sqlite3 데이터 베이스에 저장

C:\Users\hi\AppData\Local\Temp\ipykernel_14824\1822468546.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data2 = pd.read_csv(f"./data/yes24/new_yes24bestseller{year[-1]}{month[0]}_{year[0]}{month[-1]}_1000_v2.csv")


'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable


In [ ]:
urls[51]

In [ ]:
len(result)

In [ ]:
colnames = ['yy', 'mon', 'title_h', 'title_f', 'title_m', 'title_e', 'detail_link', 'author', 'publisher',
            'pub_date', 'price', 'n_reviews', 'review_link', 'rating', 'tags']

In [ ]:
df = pd.DataFrame(result, columns= colnames)

In [ ]:
df.to_csv("./data/new_yes24bestseller2023_01_1000.csv", index=False)

In [ ]:
data = pd.read_csv("./data/new_yes24bestseller2023_01_1000.csv")

In [ ]:
urls = data['detail_link']

In [ ]:
# 데이터 베이스에 저장하기
# 파일로 바로 저장하기

상세 페이지 수집

In [ ]:
detail_result.columns

In [ ]:
detail_result.info()

# 데이터 베이스에 자료 저장하기
* pandas 데이터 프레임을 DB에 바로 저장하기

In [ ]:
import sqlite3

In [ ]:
con = sqlite3.connect("sqlite3_yes24_best2023_01.db")
cursor = con.cursor()

In [ ]:
cursor.execute(f'CREATE TABLE IF NOT EXISTS yes24_best_202301({list(detail_result.columns)})')
con.commit()

In [ ]:
detail_result['page'] = detail_result['page'].apply(str)

In [ ]:
detail_result['weight'] = detail_result['weight'].apply(str)

In [ ]:
detail_result['category'] = detail_result['category'].apply(str)

In [ ]:
detail_result.to_sql('yes24_best_202301', con, if_exists='append', index=False)

In [ ]:
cursor.execute(
    '''
    SELECT * FROM yes24_best_202301 limit 10;
    ''')
for row in cursor.fetchall():
    print(row)

# sqlalchemy를 이용하는 방법

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///sqlite3_yes24_best2023_01_sqlalchemy.db', echo= False)
conn = engine.raw_connection()
test = pd.read_sql('select * from yes24_best_202301_sqlalchemy limit 3', conn)
test
conn.close()

In [ ]:
test

In [ ]:
def ramentop20todb(df, today):
    engine = create_engine("mysql+mysqldb://<id>:"+"<password>"+"@59.7.246.88:3306/ramen", encoding='utf-8')
    conn = engine.connect()
    try:
        df.to_sql(name='Top20s{}'.format(today), con=engine, if_exists='append', index=False)
    except Exception as e:
        print(e)
    conn.close()
    return print("DB saving has been completed")

In [ ]:
detail_info = pd.DataFrame()
for url2 in tqdm(urls):
    r = requests.get(url2, stream=True)
    soup = bs(r.text, 'lxml')
    book_id_num = url2.split("/")[-1]
    book_pws = soup.select_one("tbody > tr:nth-child(2) > td.txt.lastCol").get_text().split("|") if soup.select_one("tbody > tr:nth-child(2) > td.txt.lastCol") != None  else ""
    if len(book_pws) < 3:
        page = 0
        weight = 0
        size = 0
    else:
        page, weight, size = book_pws
    category = list({i.get_text() for i in soup.select("div.infoSetCont_wrap > dl:nth-child(1) > dd > ul a") if soup.select("div.infoSetCont_wrap > dl:nth-child(1) > dd > ul a") != None})
    abstract = soup.select_one(".infoWrap_txtInner").get_text() if soup.select_one(".infoWrap_txtInner") != None else ""
    pub_review = soup.select_one("#infoset_pubReivew div.infoWrap_txt").get_text() if soup.select_one("#infoset_pubReivew div.infoWrap_txt") != None else ""
    result = pd.DataFrame([[book_id_num, page, weight, size, category, abstract, pub_review]], columns=['book_id_num','page', 'weight', 'size', 'category', 'abstract', 'pub_review'])
    detail_info = pd.concat([detail_info, result])
    time.sleep(5)

In [ ]:
detail_info

In [ ]:
pd.concat()